## Strategy: 

1. We are going to write two scripts. One is going to get the url and title of the MV. Anotehr is going to write a script to collect the views of the music video from a specific time period. This should also contain the url. 
2. We are going to look at the top 5 trending on Youtube. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import random
import time
from selenium.webdriver import Firefox
from scrape_functions import (
    get_comic_title,
    find_image,
    scrape_image,
    get_prev)

In [ ]:
browser = Firefox()
url = "https://www.xkcd.com/"
browser.get(url)

In [ ]:
for i in range(100):
    title = get_comic_title(browser)
    img = find_image(browser)
    filename = title.lower().replace(' ', '_') + '.png'
    print(filename)
    scrape_image(img, filename)
    get_prev(browser)
    time.sleep(3 + random.random()*10)

In [ ]:
!ls *.png